In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time

import tensorflow as tf
from tensorflow.keras import backend, optimizers, models, Model, applications, metrics, activations, losses
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, Dropout, Flatten, Dense, Reshape, Concatenate, Conv2DTranspose, ReLU, Activation, Normalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard
import tensorflow.keras.utils as utils

In [2]:
from google.colab import drive
from google.colab import files 


drive.mount("/content/gdrive")
print(os.getcwd())
os.chdir("/content/gdrive/My Drive/AML2021/AML2021")
print(os.getcwd())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content
/content/gdrive/.shortcut-targets-by-id/1_nr6od538RmnZRE1gB6vQYFsrHaUgS19/AML2021/AML2021


# Autoencoder

In [4]:
EPOCHS = 100
INITIAL_EPOCH = 0
BATCH_SIZE = 16
INPUT_SHAPE = (299,299,1)
CLASSES = 2
OPT = optimizers.Adam(learning_rate=1e-5)
VAL_SPLITT=0.2
os.makedirs('models/CVAE', exist_ok = True) 
os.makedirs('models/CVAE/imgs', exist_ok = True)

## Model

In [ ]:
# CVAE from https://www.tensorflow.org/tutorials/generative/cvae
class CVAE(tf.keras.Model):
  def __init__(self, encoder, decoder, name):
    super(CVAE, self).__init__(name=name)
    self.encoder = encoder# Erwartet als output 2*LATENT_SIZE einmal für mu einmal für sigma
    self.decoder = decoder #Erwartet als output logits ohne sigmoid(x), d.h. activation des letzten layers ist die identität: x->x

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean # transform standard normal to normal with mean, var

  def decode(self, z):
    return self.decoder(z)

  def forward(self, x):
    mean, logvar = self.encode(x)
    z = self.reparameterize(mean, logvar)
    x_logit = self.decode(z)
    return x, x_logit, z, mean, logvar

# Encoder, Decoder from https://www.medrxiv.org/content/10.1101/2020.04.14.20065722v1.full.pdf
def blockT(i, filter, kernel, strides=(2,2), padding='same'):
  x = Conv2DTranspose(filter, kernel, strides=strides, padding=padding)(i)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  return x

def block(i, filter, kernel, strides=(2,2), padding='same'):
  x = Conv2D(filter, kernel, strides=strides, padding=padding)(i)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  return x

def create_encoder_decoder():
  x = Input(shape=INPUT_SHAPE)
  l = block(x, 64, (3,3), strides=(2,2), padding='same')
  l = block(l, 64, (3,3), strides=(1,1), padding='same')
  l = block(l, 128, (3,3), strides=(2,2), padding='same')
  l = block(l, 128, (3,3), strides=(1,1), padding='same')
  l = block(l, 256, (3,3), strides=(2,2), padding='same')
  l = block(l, 256, (3,3), strides=(1,1), padding='same')
  l = block(l, 512, (3,3), strides=(2,2), padding='same')
  l = Flatten()(l)
  o = Dense(2*81)(l) #LATENT_DIM is 9*9=81, das 2 mal steht davor weil mean und logvar predicted werden sollen und beide shape LATENT_DIM haben
  encoder = Model(x,o, name="encoder")

  z = Input(shape=(9*9,))
  l = Reshape((9,9,1))(z)
  l = blockT(l, 512, (3,3), strides=(2,2), padding='same')#18x18x512
  l = block(l, 512, (3,3), strides=(1,1), padding='same')
  l = blockT(l, 256, (3,3), strides=(2,2), padding='same')#36x36x256
  l = blockT(l, 128, (4,4), strides=(2,2), padding='valid')#74x74x128
  l = block(l, 128, (3,3), strides=(1,1), padding='same')
  l = blockT(l, 64, (3,3), strides=(2,2), padding='same')#148x148x64
  x = Conv2DTranspose(1, (5,5), strides=(2,2), padding='valid')(l)#299x299x1
  decoder = Model(z,x, name="decoder")
  
  return encoder, decoder

In [ ]:
# Loss
def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)

def compute_loss(x, x_logit, z, mean, logvar):
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)

def get_batch(batch_nr, indices):
  batch_start = batch_nr*BATCH_SIZE
  batch_end = batch_start+BATCH_SIZE
  batch_idx = indices[batch_start:batch_end]
  return x_train[batch_idx]

# Fit
def fit(model, train_indices, val_indices):
  previous_val_loss = float('inf')
  for epoch in range(INITIAL_EPOCH, EPOCHS):
    train_losses = []
    val_losses = []

    num_train_batches = int(np.ceil(train_indices.shape[0]/BATCH_SIZE))
    num_val_batches = int(np.ceil(val_indices.shape[0]/BATCH_SIZE))
    np.random.shuffle(train_indices)
    np.random.shuffle(val_indices)
    
    # training
    for batch_nr in range(num_train_batches):
      x = tf.cast(utils.normalize(get_batch(batch_nr, train_indices), axis=0), tf.float32)
      with tf.GradientTape() as tape:
        x, x_logit, z, mean, logvar = model.forward(x)
        loss = compute_loss(x, x_logit, z, mean, logvar)
      train_losses.append(loss.numpy())
      gradients = tape.gradient(loss, model.trainable_variables)
      OPT.apply_gradients(zip(gradients, model.trainable_variables))
      print(f"\r epoch {epoch}/{EPOCHS}, batch {batch_nr+1}/{num_train_batches}", end="")
    
    #validation
    for batch_nr in range(num_val_batches):
      x = tf.cast(utils.normalize(get_batch(batch_nr, train_indices), axis=0), tf.float32)
      x, x_logit, z, mean, logvar = model.forward(x)
      loss = compute_loss(x, x_logit, z, mean, logvar)
      val_losses.append(loss.numpy())

    # reporting
    avg_train_loss = sum(train_losses) / len(train_losses)
    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f", train loss= {avg_train_loss}, val loss= {avg_val_loss}" )
    with open(f"models/CVAE/log-{model.name}.csv","a+") as f:
      f.write(f"{epoch}, {avg_train_loss}, {avg_val_loss}")
    
    # saving
    if avg_val_loss < previous_val_loss:
      model.encoder.save(f'models/CVAE/best-{model.name}-Encoder.hdf5')
      model.decoder.save(f'models/CVAE/best-{model.name}-Decoder.hdf5')
      print(f"disc loss improved from {previous_val_loss} to {avg_val_loss}, best-{model.name}.hdf5 saved")
    previous_val_loss = avg_val_loss

## Training

In [ ]:
# Load Data
x_train = np.load("x_train.npy")[:,:,:,0:1]# make rgb image to grayscale, this works because the original image was greyscale
y_train = np.load("y_train.npy")

x_cov_idx = np.where(y_train[:,0]==1)[0]
x_normal_idx = np.where(y_train[:,1]==1)[0]

# train-val-split
np.random.shuffle(x_cov_idx)
train_cov_idx = x_cov_idx[int(x_cov_idx.shape[0]*VAL_SPLITT):]
val_cov_idx = x_cov_idx[:int(x_cov_idx.shape[0]*VAL_SPLITT)]

np.random.shuffle(x_normal_idx)
train_normal_idx = x_normal_idx[int(x_normal_idx.shape[0]*VAL_SPLITT):]
val_normal_idx = x_normal_idx[:int(x_normal_idx.shape[0]*VAL_SPLITT)]

print(x_train.shape)
print(train_cov_idx.shape, val_cov_idx.shape, train_normal_idx.shape, val_normal_idx.shape)

(16324, 299, 299, 1)
(6530,) (1632,) (6530,) (1632,)


In [ ]:
# Train from scratch
INITIAL_EPOCH=0
enc_cov, dec_cov = create_encoder_decoder()
cvae_cov = CVAE(enc_cov, dec_cov, name="CVAE-Cov")
fit(cvae_cov, train_cov_idx, val_cov_idx)

INITIAL_EPOCH=0
enc_norm, dec_norm = create_encoder_decoder()
cvae_normal = CVAE(enc_norm, dec_norm, name="CVAE-Normal")
fit(cvae_normal, train_normal_idx, val_normal_idx)

In [ ]:
# Continue Training
INITIAL_EPOCH=100
EPOCHS = 130
enc_cov = models.load_model('models/CVAE/best-CVAE-Cov-Encoder.hdf5')
dec_cov = models.load_model('models/CVAE/best-CVAE-Cov-Decoder.hdf5')
cvae_cov = CVAE(enc_cov, dec_cov, name="CVAE-Cov")
fit(cvae_cov, train_cov_idx, val_cov_idx)

INITIAL_EPOCH=100
EPOCHS = 130
enc_norm = models.load_model('models/CVAE/best-CVAE-Normal-Encoder.hdf5')
dec_norm = models.load_model('models/CVAE/best-CVAE-Normal-Decoder.hdf5')
cvae_normal = CVAE(enc_norm, dec_norm, name="CVAE-Normal")
fit(cvae_normal, train_normal_idx, val_normal_idx)

 epoch 100/130, batch 205/205, train loss= 38643.77099847561, val loss= 38554.52489276961
disc loss improved from inf to 38554.52489276961, best-CVAE-Cov.hdf5 saved
 epoch 101/130, batch 205/205, train loss= 38614.166577743905, val loss= 38471.56334252451
disc loss improved from 38554.52489276961 to 38471.56334252451, best-CVAE-Cov.hdf5 saved
 epoch 102/130, batch 205/205, train loss= 38593.53212652439, val loss= 38535.59237132353
 epoch 103/130, batch 205/205, train loss= 38580.03077362805, val loss= 38561.81587009804
 epoch 104/130, batch 205/205, train loss= 38584.13809070122, val loss= 38507.88334865196
disc loss improved from 38561.81587009804 to 38507.88334865196, best-CVAE-Cov.hdf5 saved
 epoch 105/130, batch 205/205, train loss= 38583.66337652439, val loss= 38440.830729166664
disc loss improved from 38507.88334865196 to 38440.830729166664, best-CVAE-Cov.hdf5 saved
 epoch 106/130, batch 205/205, train loss= 38580.41945503049, val loss= 38509.591145833336
 epoch 107/130, batch 20

# Classifier

In [8]:
INITIAL_EPOCH = 0
EPOCHS = 120
BATCH_SIZE = 16
INPUT_SHAPE = (299,299,1)
CLASSES = 2
VAL_SPLITT = 0.2
OPTIMIZER = optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False)

METRICS = [metrics.AUC(name='auroc', curve='ROC'),
           metrics.CategoricalAccuracy(name='accuracy'),
           metrics.Precision(name='precision'),
           metrics.Recall(name='recall')
           ]

LRS = ReduceLROnPlateau(monitor='val_auroc', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001)
ES = EarlyStopping(monitor='val_auroc', min_delta=0.0001, patience=9, verbose=0, mode='auto', baseline=None, restore_best_weights=False)


MC_BEST = ModelCheckpoint(f'models/CVAE/best-CLS.hdf5', monitor='val_auroc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq="epoch")
LOG = CSVLogger(f'models/CVAE/CLS.log')
TB = TensorBoard(log_dir=f'models/CVAE/logs')

In [4]:
# Load Data
x_train = np.load("x_train.npy")[:,:,:,0:1]# make rgb image to grayscale, this works because the original image was greyscale
x_normalized = np.zeros(x_train.shape, dtype="float32")
idx = np.arange(0,x_train.shape[0])
num_batches = int(np.ceil(idx.shape[0]/BATCH_SIZE))
for batch_nr in range(num_batches):
  batch_start = batch_nr*BATCH_SIZE
  batch_end = batch_start+BATCH_SIZE
  batch_idx = idx[batch_start:batch_end]
  x_normalized[batch_idx] = tf.cast(utils.normalize(x_train[batch_idx], axis=0), tf.float32)
x_train = []

y_train = np.load("y_train.npy")

In [ ]:
e_cov = models.load_model("models/CVAE/best-CVAE-Cov-Encoder.hdf5")
d_cov = models.load_model("models/CVAE/best-CVAE-Cov-Decoder.hdf5")
e_norm = models.load_model("models/CVAE/best-CVAE-Normal-Encoder.hdf5")
d_norm = models.load_model("models/CVAE/best-CVAE-Normal-Decoder.hdf5")

e_cov.trainable = False
d_cov.trainable = False
e_norm.trainable = False
d_norm.trainable = False

e_cov._name = "e_cov"
d_cov._name = "d_cov"
e_norm._name = "e_norm"
d_norm._name = "d_norm"



x = Input(shape=INPUT_SHAPE, batch_size=BATCH_SIZE)

mean_cov, _ = tf.split(e_cov(x), num_or_size_splits=2, axis=1)
x_recon_cov = d_cov(mean_cov)

mean_norm, _ = tf.split(e_norm(x), num_or_size_splits=2, axis=1)
x_recon_normal = d_norm(mean_norm)

x_0 = tf.abs(x-x_recon_cov)
x_1 = tf.abs(x-x_recon_normal)
x_2 = tf.abs(2*x-x_recon_normal)
l = tf.concat([x_0,x_1,x_2], axis=-1)# now shape should be None, 299, 299, 3

base_model = applications.ResNet50(include_top=False, weights="imagenet", input_shape=(299,299,3))
l = base_model(l)
l = GlobalAveragePooling2D(name='avg_pool')(l)
y = Dense(CLASSES, activation='softmax', name=f'fc{CLASSES}')(l)

model = Model(x,y)
model.compile(OPTIMIZER,  loss='categorical_crossentropy', metrics=METRICS)

model.fit(x=x_normalized, y=y_train, batch_size=BATCH_SIZE, validation_split = VAL_SPLITT, callbacks = [MC_BEST, LRS, ES, LOG, TB], epochs = EPOCHS, initial_epoch = INITIAL_EPOCH)
x_train=[]
y_train=[]

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/120


In [ ]:
# Eval Model
x_test = tf.cast(utils.normalize(np.load("x_test.npy")[:,:,:,0:1], axis=0), tf.float32)
y_test = np.load("y_test.npy")

model = models.load_model(f'models/CVAE/best-CLS.hdf5', compile=True)
out = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)

# Save Results
with open("results.txt","a+") as f:
  f.write(f"CVAE-CLS test_loss: {out[0]}, test_auroc: {out[1]}, test_acc: {out[2]}, test_prec: {out[3]}, test_rec: {out[4]}, test_f1: {2* ((out[3]*out[4]) / (out[3]+out[4]))}\n")
print(f"wrote \"best-CLS test_loss: {out[0]}, test_auroc: {out[1]}, test_acc: {out[2]}, test_prec: {out[3]}, test_rec: {out[4]}, test_f1: {2* ((out[3]*out[4]) / (out[3]+out[4]))}\n\" to results.txt ")